### Target audiences for the async feature in Python
- End-user developers
- Framework developers

Much of the confusion around asyncio is due to lack of understanding of the difference in what those two groups require. For example, official Python documentation for asyncio is more appropriate for framework developers than end users. 

### QuickStart

You only need to know about seven functions to use Asyncio for everyday use. The following are a smal subset of the whole asyncio API. 

- Starting the asyncio **eventloop**
- Calling ***async/await*** functions
- Creating a **task** to be run on the loop
- Waiting for multiple tasks to complete
- Closing the loop after all concurrent tasks have completed


In [8]:
%pycat helloworld.py
    

#The "Hello World" of Asyncio

import asyncio,time

async def main():
    print(f"{time.ctime()} Hello!")
    await asyncio.sleep(1.0)
    print(f"{time.ctime()} Goodbye!")

asyncio.run(main())


Result:<br>
Sun Mar 13 15:47:37 2022 Hello!<br>
Sun Mar 13 15:47:38 2022 Goodbye!<br>

#### run()
Asyncio-based code will use run() function to execute an async def function. <br>

That being said, however it's important to understand what that function is doing for you.<br>
Let me introduce the ideas that we'll build on throughout the rest of this tutorial. 

In [10]:
#hello-ish world 

%pycat hello_ish_world.py

import asyncio , time

async def main():
    print(f"{time.ctime()} Hello!")
    await asyncio.sleep(1.0)
    print(f"{time.ctime()} Goodbye!")

loop = asyncio.get_event_loop()  # Create eventloop instance
task = loop.create_task(main())  # schedules your coroutine to be run on the loop.
loop.run_until_complete(task)    # block the current thread

pending = asyncio.all_tasks(loop=loop) #Return a set of not yet finished Task objects run by the loop.
for task in pending: #gather still pending tasks 
    task.cancel()    # and cancel them 

group = asyncio.gather(*pending,return_exceptions=True) #method for gathering 
loop.run_until_complete(group)   #run them again 

loop.close()                     # It must be called and it will clear all queues and shutdown executor


#### loop = asyncio.get_event_loop()
This is how you get a loop instance. <br>
If you are inside an **async function**, you should call asyncio.get_running_loop() instad.


#### task = loop.create_task(coro)
Your coroutin function will not be executed until you schedules them to the loop.<br>
The returned object, task, can be used to monitor the status of the task.<br>
You can also cancel them with **task.cancel()**.


#### loop.run_until_complete(coro)
This call will block the current thread(usually main thread). In this case, we put "task" object we got previously. So run_until_complete() will keep the loop running only until the given task completes -- all the other tasks scheduled on the loop will also run while the loop is running.<br><br> 

    Q. If the loop stops running as the task object given completes, does in go into pending state?

Internally, asyncio.run() calls **run_until_complete()** for you. 

#### group=asyncio.gather(task1, task2, task3)
To gather a still-pending you get all tasks by:

    pending = asyncio.all_tasks(loop=loop)

And cancel the tasks by:

    for task in pending: 
        task.cancel() 

And gather it by:

    group = asyncio.gather(*pending,return_exceptions=True)

and run all the tasks that are pending by:

    loop.run_until_complete(group)

Interally, asyncio.ru() will do all of the cancelling, gathering, and waiting for pending tasks to funish up.

#### loop.close()
This will clear all queues and shut down the executor. A *stopped* loop can be restarted but a *closed* loop is gone for good. Internally, asyncio.run() will close the loop before returning. 


### Last item of basic functionality : how to run ***Blocking*** functions
The thing about cooperatieve multitasking is allow a context switch back to the loop using the keyword ***await***. The problem is, most Python code in the wild doesn't do this but instead may require you to run such functions in threads. So, sometimes using such blocking libraries is unavoiable. <br><br>

For that, asyncio provides an API that is very similar to API in the ***concurent.futures*** package. There are a couple of quirks to be aware of. 

In [11]:
#quickstart_exe.py

%pycat quickstart_exe.py

import time
import asyncio

async def main():
    print(f"{time.ctime()} Hello!")
    asyncio.sleep(1.0)
    print(f"{time.ctime()} Goodbye!")
    
def blocking():
    time.sleep(0.5)
    print(f"{time.ctime()} Hello from a thread!")

loop = asyncio.get_event_loop()
task = loop.create_task(main())
loop.run_in_executor(None, blocking)
loop.run_until_complete(task)

pending= asyncio.all_tasks(loop=loop)
for task in pending:
    task.cancel()

group= asyncio.gather(*pending, return_exceptions=True)
loop.run_until_complete(group)
loop.close()


#### blocking()
it calls traditional time.sleep() and this would have blocked the main thread nad prevented your event loop from running. So, you must not make this funcition a coroutine. We solve this problem by running this function in an executor by:

    loop.run_in_executor(None, blocking)

#### loop.run_in_executor(None, blocking)
Sometimes you need to run things in a separte thread or even a separate process: this method is used for that. Here we pass blocking function to be run in the default executor. Note that **run_in_executor()** doesn't block the main thread; it only schedules the executor task to run and returns ***Future*** object. The executor task will begin executing **ONLY AFTER run_until_complete()** is called. 


#### pending = asyncio.all_tasks(loop=loop)
The set of tasks in pending does NOT include an entry for the call to **blocking()** made in run_in_executor(). This will be true of any call that returns **Future** rather than a **Task**. Just remember that **all_tasks()** really returns only Tasks, not Futures. 


#### Output of running the script

    python quickstart_exe.py
    Sun Mar 13 16:57:56 2022 Hello!
    Sun Mar 13 16:57:57 2022 Hello from a thread!
    Sun Mar 13 16:57:57 2022 Goodbye!

